## Inspect results of motion correction
Step 2 of the Caiman processing pipeline for multi-layer two-photon calcium imaging movies. This notebook shows the results of the motion correction performed in step 1 and allows the selection of 'bad frames' (i.e. frames which have too much residual motion). This is an interactive step that has to be run seperately for each dataset.

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# Import Bokeh library
from bokeh.plotting import Figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
from bokeh.io import output_notebook, export_svgs
from bokeh.models.sources import ColumnDataSource
from bokeh import palettes

In [ ]:
# This has to be in a separate cell, otherwise it wont work.
from bokeh import resources
output_notebook(resources=resources.INLINE)

### Display average signal intensity
This step is optional and useful as sanity check how the imported data looks like.

In [ ]:
# select group (0, 1, ...)
group_ix = 0

color_map = palettes.d3['Category10'][10] # colors for different planes

# prepare data structure
trial_names = [x.replace(working_data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]
data = {'x': np.array(range(total_frames_list[group_ix])), 
        'trial_idx': trial_indices_list[group_ix],
        'trial_name': trial_names_frames
       }

# add average for each plane
for i_plane in range(n_planes):
    tiff_file = os.path.join(working_data_folder, joined_tif_list[group_ix] + '_P%d.tif' % (i_plane))
    mov = cm.load(tiff_file, outtype=np.int16)

    # plot average signal intensity per frame
    frame_avg = np.mean(np.mean(mov, axis=1), axis=1)

    fieldname = 'y%s' % (i_plane)
    data[fieldname] = frame_avg
    
data_source = ColumnDataSource(data)

# create figure and plot
p = Figure(plot_width=900, plot_height=300, title=('Frame average - Group %d' % (group_ix))) 
p.add_tools(CrosshairTool(), utils.getHover())
for i_plane in range(n_planes):
    p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

show(p)

### Correlation with template - uncorrected data
First, we just get the correlation with the template for the uncorrected data. This part is optional and can be used for example to re-assign groups.

In [ ]:
# Todo: parallelize
corr_orig_list = []

t_start = time.time()
for ix_file, i_file in enumerate(joined_tif_list):
    corr_orig_list.append([])
    for i_plane in range(n_planes):
        fname = os.path.join(working_data_folder, i_file + '_P%d.tif' % (i_plane))

        # compute initial template by binned median filtering
        template = cm.load(fname).bin_median(window=10)
    
        corr_orig = []
        for frame in cm.load(fname):
            corr_orig.append(scipy.stats.pearsonr(frame.flatten(), template.flatten())[0])
        corr_orig_list[ix_file].append(corr_orig)
    
# print elapsed time
t_elapsed = time.time() - t_start
print('\nFinished pre-MC in %1.2f s (%1.2f s per frame)' % (t_elapsed, t_elapsed/(sum(total_frames_list)*n_planes)))

Plot results of the correlation with template.

In [ ]:
# select group (0, 1, ...)
group_ix = 0

trial_names = [x.replace(working_data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]

# frames = np.array(range(len(corr_orig_list[group_ix])))
data = {'x': np.array(range(len(corr_orig_list[group_ix][0]))), 
        'trial_idx': trial_indices_list[group_ix],
        'trial_name': trial_names_frames
       }

# add average for each plane
for i_plane in range(n_planes):
    fieldname = 'y%s' % (i_plane)
    data[fieldname] = corr_orig_list[group_ix][i_plane]

data_source = ColumnDataSource(data)

p = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d' % (group_ix))) 
p.add_tools(CrosshairTool(), utils.getHover())
for i_plane in range(n_planes):
    p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

show(p)

### Metrics and summary plots
Print different metrics for raw movie and rigid / pw-rigid corrected movies.

In [ ]:
for i_group in range(n_groups):
    for i_plane in range(n_planes):
        print('MC evaluation - Group %d - Plane %d:' % (i_group, i_plane))
        mc_utils.printMetrics(corr_mean[i_group][i_plane], corr_min[i_group][i_plane], crispness[i_group][i_plane], norms[i_group][i_plane])
        print('\n')

Plot correlations of each frame with the template image (binned median) for original, rigid correction and pw-rigid correction. The bokeh plotting library provides a toolbar for interaction with the plot.

In [ ]:
# select group (0, 1, ...)
group_ix = 0
# select plane (0, 1, ..)
plane_ix = 0

p1 = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d - Plane %d' % (group_ix, plane_ix))) 
frames = np.array(range(len(metrics[group_ix][plane_ix]['corr_orig'])))
p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_orig']), line_width=2, legend='Original', color='blue')
p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_rig']), line_width=2, legend='Rigid', color='orange')
if opts.get('motion', 'pw_rigid'):
    p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_els']), line_width=2, legend='PW-Rigid', color='green')

p2 = Figure(plot_width=250, plot_height=250)
p2.circle(np.array(metrics[group_ix][plane_ix]['corr_orig']), np.array(metrics[group_ix][plane_ix]['corr_rig']), size=5)
p2.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
p2.xaxis.axis_label = 'Original'
p2.yaxis.axis_label = 'Rigid'

if opts.get('motion', 'pw_rigid'):
    p3 = Figure(plot_width=250, plot_height=250)
    p3.circle(np.array(metrics[group_ix][plane_ix]['corr_rig']), np.array(metrics[group_ix][plane_ix]['corr_els']), size=5)
    p3.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
    p3.xaxis.axis_label = 'Rigid'
    p3.yaxis.axis_label = 'PW-Rigid'

# make a grid
if opts.get('motion', 'pw_rigid'):
    grid = gridplot([[p1, None], [p2, p3]], sizing_mode='fixed', toolbar_location='left')
else:
    grid = gridplot([[p1, p2], [None, None]], sizing_mode='fixed', toolbar_location='left')

show(grid)

### Detect frames with bad motion
Identify frames with significant residual motion (low correlation with template). Write a JSON file with criterion and indices of frames matching the criterion. This file can be used in further analysis to exclude the frames corrupted by motion.

In [ ]:
thresh = [
    [0, 0.1, 0.1, 0.1]
] # find frames where value is less than criterion (one value per group and plane)

for i_group in range(n_groups):
    for i_plane in range(n_planes):
        print('Group %d - Plane %d' % (i_group, i_plane))
        if opts.get('motion', 'pw_rigid'):
            # pw-rigid registration
            criterion = 'corr_els'
            bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) 
                          if i < thresh[i_group][i_plane]]
            print('%1.0f frames matching criterion after pw-rigid registration.' % (len(bad_frames)))
            mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], 
                                        bad_frames, mc_list[i_group][i_plane], 'els', working_data_folder)
        # rigid registration
        criterion = 'corr_rig'
        bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) 
                      if i < thresh[i_group][i_plane]]
        print('\n%1.0f frames matching criterion after rigid registration.' % (len(bad_frames)))
        mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], 
                                    bad_frames, mc_list[i_group][i_plane], 'rig', working_data_folder)
        print('\n')